In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

import numpy.random as nr
import matplotlib.pyplot as plt

import scipy.stats as ss
import math

In [2]:


# Load the dataset
df = pd.read_csv('wages.csv')

# Display the first few rows of the DataFrame to ensure it's loaded correctly
df.head()


,logwage,hgc,college,tenure,age,married
0,NaN,12.0,not college grad,5.333333,37,single
1,1.856449,12.0,not college grad,5.250000,37,single
2,1.612777,12.0,not college grad,1.250000,42,single
3,2.200974,17.0,college grad,1.750000,43,married
4,2.089854,12.0,not college grad,17.750000,42,married


In [3]:
# Applying Label Encoding to 'college'
df['college_encoded'] = pd.factorize(df['college'])[0]

# Applying One-Hot Encoding to 'married'
df['married_encoded'] = pd.factorize(df['married'])[0]

# Display the modified DataFrame
print(df.head())

    logwage   hgc           college     tenure  age  married  college_encoded  \
0       NaN  12.0  not college grad   5.333333   37   single                0   
1  1.856449  12.0  not college grad   5.250000   37   single                0   
2  1.612777  12.0  not college grad   1.250000   42   single                0   
3  2.200974  17.0      college grad   1.750000   43  married                1   
4  2.089854  12.0  not college grad  17.750000   42  married                0   

   married_encoded  
0                0  
1                0  
2                0  
3                1  
4                1  


In [4]:
df.shape

(2246, 8)

In [5]:
def quality_report(df):
    """
    Description: Displays quality of data in terms of missing values, 
    unique numbers, datatypes etc.
    
    Arguments: Dataframe
    """
    dtypes = df.dtypes
    nuniq = df.T.apply(lambda x: x.nunique(), axis=1)
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    quality_df  = pd.concat([total, percent, nuniq, dtypes], axis=1, keys=['Total NaN', 'Percent of NaN','Nunique', 'Dtype'])
    display(quality_df)


In [6]:
quality_report(df)

,Total NaN,Percent of NaN,Nunique,Dtype
logwage,560,24.933215,674,float64
tenure,15,0.667854,259,float64
hgc,2,0.089047,16,float64
college,0,0.000000,2,object
age,0,0.000000,13,int64
married,0,0.000000,2,object
college_encoded,0,0.000000,2,int64
married_encoded,0,0.000000,2,int64


# Drop Observations Where hgc or tenure Are Missing

### I use the dropna() function from pandas, specifying hgc and tenure in the subset argument to drop rows where these specific columns have missing values.

In [7]:
# Drop rows where `hgc` or `tenure` are missing
df = df.dropna(subset=['hgc', 'tenure'])

# Display the shape of the DataFrame to check how many rows are left
df.shape


(2229, 8)

In [8]:
quality_report(df)

,Total NaN,Percent of NaN,Nunique,Dtype
logwage,560,25.123374,669,float64
hgc,0,0.000000,16,float64
college,0,0.000000,2,object
tenure,0,0.000000,259,float64
age,0,0.000000,13,int64
married,0,0.000000,2,object
college_encoded,0,0.000000,2,int64
married_encoded,0,0.000000,2,int64


# Using modelsummary to Produce a Summary Table

In [9]:
# Generate descriptive statistics
summary_table = df.describe()
summary_table

,logwage,hgc,tenure,age,college_encoded,married_encoded
count,1669.000000,2229.000000,2229.000000,2229.000000,2229.000000,2229.000000
mean,1.625190,13.101391,5.970615,39.151638,0.237775,0.641992
std,0.385534,2.524306,5.507216,3.061954,0.425816,0.479522
min,0.004940,0.000000,0.000000,34.000000,0.000000,0.000000
25%,1.362255,12.000000,1.583333,36.000000,0.000000,0.000000
50%,1.655079,12.000000,3.750000,39.000000,0.000000,1.000000
75%,1.936200,15.000000,9.333333,42.000000,0.000000,1.000000
max,2.261495,18.000000,25.916670,46.000000,1.000000,1.000000


In [10]:
# Convert the descriptive statistics to LaTeX format
summary_latex = summary_table.to_latex()

In [11]:
# Write the LaTeX string to a file
with open('C:/Users/OLUBAYODE/Documents/Data Science Economics/summary_latex', 'w') as file:
    file.write(summary_latex)

# Confirmation message
print("Descriptive statistics have been exported to 'summary_latex.tex'.")

Descriptive statistics have been exported to 'summary_latex.tex'.


In [12]:
# Check the first few lines of the LaTeX string to ensure it looks correct
print("\n".join(summary_latex.split("\n")[:10]))

\begin{tabular}{lrrrrrr}
\toprule
{} &      logwage &          hgc &       tenure &          age &  college\_encoded &  married\_encoded \\
\midrule
count &  1669.000000 &  2229.000000 &  2229.000000 &  2229.000000 &      2229.000000 &      2229.000000 \\
mean  &     1.625190 &    13.101391 &     5.970615 &    39.151638 &         0.237775 &         0.641992 \\
std   &     0.385534 &     2.524306 &     5.507216 &     3.061954 &         0.425816 &         0.479522 \\
min   &     0.004940 &     0.000000 &     0.000000 &    34.000000 &         0.000000 &         0.000000 \\
25\%   &     1.362255 &    12.000000 &     1.583333 &    36.000000 &         0.000000 &         0.000000 \\
50\%   &     1.655079 &    12.000000 &     3.750000 &    39.000000 &         0.000000 &         1.000000 \\


# Analyze Missingness of Log Wages

### To analyze the rate at which log wages are missing and theorize whether the missingness is MCAR (Missing Completely At Random), MAR (Missing At Random), or MNAR (Missing Not At Random), I will first calculate the missing rate.

In [13]:
# Calculate the rate of missing values for log wages
missing_rate_logwage = df['logwage'].isnull().mean()

print(f"Rate of missing log wages: {missing_rate_logwage:.2%}")


Rate of missing log wages: 25.12%


In [14]:
# Create a boolean mask indicating where 'logwage' is missing
missing_logwage = df['logwage'].isnull()

# Group the data based on the presence or absence of 'logwage' data
# Calculate the mean for each group
grouped_means = df.groupby(missing_logwage).mean()
print(grouped_means)

         logwage        hgc    tenure        age  college_encoded  \
logwage                                                             
False    1.62519  12.556022  5.225235  39.171360         0.153984   
True         NaN  14.726786  8.192113  39.092857         0.487500   

         married_encoded  
logwage                   
False           0.654284  
True            0.605357  


### The mean values of other variables grouped by the presence or absence of logwage data reveal some differences. For example, the average years of education (hgc) and tenure at the current job are higher in rows where logwage is missing. This suggests that the missingness of logwage may not be completely random (MCAR) but could be related to other observed variables, hinting at a potential MAR situation or MNAR.

## Consideration the context and nature of the data, a deeper understanding emerges, suggesting that the missingness of logwage data is, in fact, Missing Not At Random (MNAR). The dataset comprises wage information, a sensitive topic where the likelihood of non-disclosure could be influenced by the amount of wage itself. Both higher and lower earners may have distinct motivations for not disclosing their income, with higher earners possibly concerned about privacy or social scrutiny, and lower earners potentially influenced by social stigma or personal dissatisfaction. This selective non-disclosure directly relates to the values of logwage itself, making the missing data MNAR

In [15]:
import statsmodels.formula.api as smf
import numpy as np

# Drop rows where `logwage` is missing
complete_cases_df = df.dropna(subset=['logwage'])

# Define the regression formula
formula = 'logwage ~ hgc + C(college) + tenure + np.power(tenure, 2) + age + C(married)'

# Estimate the linear regression model
model_complete_cases = smf.ols(formula, data=complete_cases_df).fit()


In [16]:
# Print the summary of the model
model_complete_cases.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                logwage   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     72.92
Date:                Thu, 21 Mar 2024   Prob (F-statistic):           7.03e-81
Time:                        00:14:24   Log-Likelihood:                -581.94
No. Observations:                1669   AIC:                             1178.
Df Residuals:                    1662   BIC:                             1216.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.5336      0.146      3.656      0.000       0.247       0.820
C(college)[T.not college grad]     0.1452      0.034      4.210      0.000       0.078       0.213
C(married)[T.single]              -0.0220      0.018     -1.245      0.213      -0.057       0.013
hgc                                0.0624      0.005     11.588      0.000       0.052       0.073
tenure                             0.0495      0.005      9.558      0.000       0.039       0.060
np.power(tenure, 2)               -0.0016      0.000     -5.324      0.000      -0.002      -0.001
age                                0.0004      0.003      0.161      0.872      -0.005       0.006
==============================================================================
Omnibus:                       99.230   Durbin-Watson:                   1.864
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              124.898
Skew:                          -0.563   Prob(JB):                     7.56e-28
Kurtosis:                       3.727   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Perform Mean Imputation to Fill in Missing Log Wages
### For mean imputation, I'll fill in missing logwage observations with the mean value of the logwage column from the complete cases.

In [17]:
# Calculate the mean of `logwage` from complete cases
logwage_mean = complete_cases_df['logwage'].mean()

# Perform mean imputation
df['logwage_mean_imputed'] = df['logwage'].fillna(logwage_mean)

# Estimate the regression with mean imputed logwage
model_mean_imputed = smf.ols(formula.replace('logwage', 'logwage_mean_imputed'), data=df).fit()


In [18]:
model_mean_imputed.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     logwage_mean_imputed   R-squared:                       0.147
Model:                              OLS   Adj. R-squared:                  0.145
Method:                   Least Squares   F-statistic:                     63.97
Date:                  Thu, 21 Mar 2024   Prob (F-statistic):           1.77e-73
Time:                          00:14:24   Log-Likelihood:                -537.58
No. Observations:                  2229   AIC:                             1089.
Df Residuals:                      2222   BIC:                             1129.
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.7076      0.116      6.111      0.000       0.481       0.935
C(college)[T.not college grad]     0.1682      0.026      6.550      0.000       0.118       0.219
C(married)[T.single]              -0.0268      0.014     -1.966      0.049      -0.054    -7.4e-05
hgc                                0.0497      0.004     11.427      0.000       0.041       0.058
tenure                             0.0382      0.004      9.813      0.000       0.031       0.046
np.power(tenure, 2)               -0.0013      0.000     -6.354      0.000      -0.002      -0.001
age                                0.0002      0.002      0.093      0.926      -0.004       0.004
==============================================================================
Omnibus:                      104.284   Durbin-Watson:                   1.914
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              160.943
Skew:                          -0.407   Prob(JB):                     1.13e-35
Kurtosis:                       4.035   Cond. No.                     2.22e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Impute Missing Log Wages as Their Predicted Values from the Complete Cases Regression
## I'll use the model estimated from the complete cases to predict logwage for the missing observations.

In [19]:
# Predict logwage using the complete cases model for all observations
df['logwage_predicted'] = model_complete_cases.predict(df)

# Use predicted logwage for missing values
df['logwage_pred_imputed'] = df['logwage'].fillna(df['logwage_predicted'])

# Estimate the regression with predicted imputed logwage
model_pred_imputed = smf.ols(formula.replace('logwage', 'logwage_pred_imputed'), data=df).fit()


In [20]:
df['logwage_predicted'] 

0       1.641504
1       1.638752
2       1.483410
3       1.695118
4       1.833661
          ...   
2241    1.517260
2242    1.922953
2243    1.777361
2244    1.796376
2245    1.575385
Name: logwage_predicted, Length: 2229, dtype: float64

In [21]:
df['logwage_pred_imputed'] 

0       1.641504
1       1.856449
2       1.612777
3       2.200974
4       2.089854
          ...   
2241    1.707857
2242    1.922953
2243    1.341422
2244    0.895134
2245    1.968204
Name: logwage_pred_imputed, Length: 2229, dtype: float64

In [22]:
model_pred_imputed.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     logwage_pred_imputed   R-squared:                       0.277
Model:                              OLS   Adj. R-squared:                  0.275
Method:                   Least Squares   F-statistic:                     141.7
Date:                  Thu, 21 Mar 2024   Prob (F-statistic):          2.33e-152
Time:                          00:14:24   Log-Likelihood:                -454.74
No. Observations:                  2229   AIC:                             923.5
Df Residuals:                      2222   BIC:                             963.4
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.5336      0.112      4.783      0.000       0.315       0.752
C(college)[T.not college grad]     0.1452      0.025      5.866      0.000       0.097       0.194
C(married)[T.single]              -0.0220      0.013     -1.677      0.094      -0.048       0.004
hgc                                0.0624      0.004     14.892      0.000       0.054       0.071
tenure                             0.0495      0.004     13.215      0.000       0.042       0.057
np.power(tenure, 2)               -0.0016      0.000     -7.734      0.000      -0.002      -0.001
age                                0.0004      0.002      0.212      0.832      -0.004       0.005
==============================================================================
Omnibus:                      238.275   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              520.308
Skew:                          -0.651   Prob(JB):                    1.04e-113
Kurtosis:                       4.977   Cond. No.                     2.22e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
import numpy as np

# Prepare the data: extract relevant features and target for imputation purposes
# df['college_encoded'] = df['college'].apply(lambda x: 1 if x == 'college grad' else 0)
# df['married_encoded'] = df['married'].apply(lambda x: 1 if x == 'married' else 0)

features_for_imputation = df[['logwage', 'hgc', 'tenure', 'age', 'college_encoded', 'married_encoded']]


# features_for_imputation = df[['logwage', 'hgc', 'tenure', 'age']]  # Example features

# Initialize the IterativeImputer
imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0)

# Perform multiple imputation
features_imputed = imputer.fit_transform(features_for_imputation)

# Extract the imputed logwage values
df['logwage_multi_imputed'] = features_imputed[:, 0]  # Assuming 'logwage' is at index 0

# Adjust the regression formula to use the multi-imputed 'logwage'
formula_multi_imputed = formula.replace('logwage', 'logwage_multi_imputed')

# Estimate the regression model with the multi-imputed 'logwage'
model_multi_imputed = smf.ols(formula_multi_imputed, data=df).fit()


In [24]:
model_multi_imputed.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     logwage_multi_imputed   R-squared:                       0.277
Model:                               OLS   Adj. R-squared:                  0.275
Method:                    Least Squares   F-statistic:                     142.1
Date:                   Thu, 21 Mar 2024   Prob (F-statistic):          9.41e-153
Time:                           00:14:24   Log-Likelihood:                -461.68
No. Observations:                   2229   AIC:                             937.4
Df Residuals:                       2222   BIC:                             977.3
Df Model:                              6                                         
Covariance Type:               nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.5672      0.112      5.068      0.000       0.348       0.787
C(college)[T.not college grad]     0.1397      0.025      5.629      0.000       0.091       0.188
C(married)[T.single]              -0.0213      0.013     -1.615      0.106      -0.047       0.005
hgc                                0.0612      0.004     14.570      0.000       0.053       0.069
tenure                             0.0411      0.004     10.922      0.000       0.034       0.048
np.power(tenure, 2)               -0.0010      0.000     -4.942      0.000      -0.001      -0.001
age                                0.0004      0.002      0.209      0.835      -0.004       0.005
==============================================================================
Omnibus:                      232.444   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              492.898
Skew:                          -0.646   Prob(JB):                    9.30e-108
Kurtosis:                       4.907   Cond. No.                     2.22e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
betas = {
    'Complete Cases': model_complete_cases.params['hgc'],
    'Mean Imputation': model_mean_imputed.params['hgc'],
    'Predicted Imputation': model_pred_imputed.params['hgc'],
    'Multiple Imputation': model_multi_imputed.params['hgc'], # Placeholder for multiple imputation model
}

betas


{'Complete Cases': 0.062393124046609724,
 'Mean Imputation': 0.04968765088652512,
 'Predicted Imputation': 0.062393124046609225,
 'Multiple Imputation': 0.06123508685002646}

In [26]:
betas_df = pd.DataFrame({
    'Complete Cases': model_complete_cases.params['hgc'],
    'Mean Imputation': model_mean_imputed.params['hgc'],
    'Predicted Imputation': model_pred_imputed.params['hgc'],
    'Multiple Imputation': model_multi_imputed.params['hgc']  # Placeholder for multiple imputation model
}, index=['hgc'])  # Specify the index here

betas_df


,Complete Cases,Mean Imputation,Predicted Imputation,Multiple Imputation
hgc,0.062393,0.049688,0.062393,0.061235


In [27]:
# Convert the DataFrame to LaTeX
betas_latex = betas_df.to_latex()

# Optionally, you can write this string to a .tex file
with open('model_betas.tex', 'w') as f:
    f.write(betas_latex)

In [28]:
betas_df

,Complete Cases,Mean Imputation,Predicted Imputation,Multiple Imputation
hgc,0.062393,0.049688,0.062393,0.061235


In [29]:
import pandas as pd

# Assuming your models are named as follows:
# model_complete_cases, model_mean_imputed, model_pred_imputed, model_multi_imputed

# Create a DataFrame to hold the coefficients
coefficients_df = pd.DataFrame({
    'Complete Cases': model_complete_cases.params,
    'Mean Imputation': model_mean_imputed.params,
    'Predicted Imputation': model_pred_imputed.params,
    'Multiple Imputation': model_multi_imputed.params  # Assuming this model exists and has been fitted
})#.fillna(0)  # Fill NaNs with 0s for any missing coefficients across models for comparison


In [30]:
coefficients_df

,Complete Cases,Mean Imputation,Predicted Imputation,Multiple Imputation
Intercept,0.533569,0.707596,0.533569,0.567189
C(college)[T.not college grad],0.145168,0.168228,0.145168,0.139741
C(married)[T.single],-0.022046,-0.026833,-0.022046,-0.021299
hgc,0.062393,0.049688,0.062393,0.061235
tenure,0.049525,0.038168,0.049525,0.041061
"np.power(tenure, 2)",-0.001560,-0.001330,-0.001560,-0.001000
age,0.000441,0.000200,0.000441,0.000435


In [31]:
# Convert the DataFrame to LaTeX
coefficients_latex = coefficients_df.to_latex()

# Optionally, you can write this string to a .tex file
with open('model_coefficients.tex', 'w') as f:
    f.write(coefficients_latex)


In [ ]:
Based on the regression results for the hgc variable (which we're interpreting as β1) across four models, I can analyze the impact of different imputation methods on the estimated returns to schooling hgc (i.e., the effect of an additional year of education on log wages). Here's a detailed look at the hgc coefficient across the models:

Complete Cases: β1 = 0.062393
Mean Imputation: β1 = 0.049688
Predicted Imputation: β1 = 0.062393
Multiple Imputation: β1 = 0.061235
The true value of β1 is given as 0.093, which serves as a benchmark for evaluating the accuracy of our estimates.

Analysis of β1 Across Models
Complete Cases and Predicted Imputation: Both methods yield a β1 of 0.062393, which is lower than the true value of 0.093. The identical estimates suggest that the predicted imputation method (which uses the complete cases regression to predict missing logwage values) does not significantly alter the estimate for hgc compared to using only complete cases. This could indicate that the mechanism of missingness in logwage does not heavily bias the estimate of hgc when using these methods.

Mean Imputation: This method yields the lowest β1 estimate of 0.049688, significantly deviating from the true value. Mean imputation tends to reduce the variability in the imputed variable (logwage in this case) and can lead to biased estimates especially if the missing data are not Missing Completely At Random (MCAR). This result suggests that mean imputation might be the least reliable method for this specific context, potentially underestimating the true effect of education on wages.

Multiple Imputation: The estimate for β1 using multiple imputation is 0.061235, slightly lower than the complete cases and predicted imputation methods but closer to them than to the mean imputation estimate. Multiple imputation generally provides a more nuanced handling of missing data by creating multiple complete datasets and pooling the results, which can help address the biases associated with simpler imputation methods.

Conclusions
The complete cases and predicted imputation methods provide higher estimates for β1 compared to mean imputation, suggesting they may be more reliable in this context, although they still underestimate the true effect of education on wages compared to the given true value of 0.093.
Mean imputation significantly underestimates the returns to schooling, likely due to its simplistic assumption about missing data, which can introduce bias.
Multiple imputation offers a balanced estimate, reflecting a sophisticated approach to handling missing data. However, it still falls short of the true value, which may suggest the presence of other factors not captured in the model or that the missing data mechanism affects the logwage variable in a way that even sophisticated imputation cannot fully correct for.
This analysis highlights the importance of choosing appropriate imputation methods based on the nature of the missing data and the specific context of the analysis. While multiple imputation generally provides a robust approach, understanding the limitations of each method is crucial for accurate statistical inference.